In [19]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [20]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [21]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# Show DataFrame
df.show(truncate=False)

+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|id                                  |date      |date_built|price |bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d6-9c31-7398aa8f6089|2022-04-08|2016      |936923|4       |3        |3167       |11733   |2     |1         |76  |
|7530a2d8-1ae3-4517-9f4a-befe060c4353|2021-06-13|2013      |379628|2       |2        |2235       |14384   |1     |0         |23  |
|43de979c-0bf0-4c9f-85ef-96dc27b258d5|2019-04-12|2014      |417866|2       |2        |2127       |10575   |2     |0         |0   |
|b672c137-b88c-48bf-9f18-d0a4ac62fb8b|2019-10-16|2016      |239895|2       |2        |1631       |11149   |2     |0         |0   |
|e0726d4d-d595-4074-8283-4139a54d0d63|2022-01-08|2017      |424418|3       |2      

In [22]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("Home_sales_df")

In [23]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
sql="""
select YEAR(date)as YEAR,
ROUND(AVG(price),2)as PRICE from Home_sales_df
WHERE bedrooms =4
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(sql).show()

+----+---------+
|YEAR|    PRICE|
+----+---------+
|2022|296363.88|
|2021|301819.44|
|2020|298353.78|
|2019| 300263.7|
+----+---------+



In [24]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
sql="""
select YEAR(date_built)as YEARBUILD,
ROUND(AVG(price),2)as PRICE from Home_sales_df
WHERE bedrooms =3 AND bathrooms =3
GROUP BY YEARBUILD
ORDER BY YEARBUILD DESC
"""
spark.sql(sql).show()

+---------+---------+
|YEARBUILD|    PRICE|
+---------+---------+
|     2017|292676.79|
|     2016|290555.07|
|     2015| 288770.3|
|     2014|290852.27|
|     2013|295962.27|
|     2012|293683.19|
|     2011|291117.47|
|     2010|292859.62|
+---------+---------+



In [25]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
sql="""
select YEAR(date_built)as YEARBUILD,
ROUND(AVG(price),2) as PRICE from Home_sales_df
WHERE bedrooms =3 AND bathrooms =3 AND floors =2 AND sqft_living >=2000
GROUP BY YEARBUILD
ORDER BY YEARBUILD DESC
"""
spark.sql(sql).show()


+---------+---------+
|YEARBUILD|    PRICE|
+---------+---------+
|     2017|280317.58|
|     2016| 293965.1|
|     2015|297609.97|
|     2014|298264.72|
|     2013|303676.79|
|     2012|307539.97|
|     2011|276553.81|
|     2010|285010.22|
+---------+---------+



In [42]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
sql="""
SELECT view,
ROUND(AVG(price), 2) AS avg_price
FROM Home_sales_df
GROUP BY view HAVING AVG(price) >= 350000 ORDER BY view DESC.show
ORDER BY view DESC;

"""
start_time = time.time()

print("--- %s seconds ---" % (time.time() - start_time))

--- 7.772445678710938e-05 seconds ---


In [43]:
# 7. Cache the the temporary table home_sales_df.
spark.sql("cache table Home_sales_df")


DataFrame[]

In [44]:
# 8. Check if the table is cached.
spark.catalog.isCached('Home_sales_df')

True

In [45]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.
sql="""
SELECT view,
ROUND(AVG(price), 2) AS avg_price
FROM Home_sales_df
GROUP BY view HAVING AVG(price) >= 350000 view DESC.show
ORDER BY view;
start_time = time.time()

"""

print("--- %s seconds ---" % (time.time() - start_time))


--- 9.12803316116333 seconds ---


In [46]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("delayed_partitioned")


In [47]:
# 11. Read the parquet formatted data.
df.write.parquet('parquet_delay_basic', mode='overwrite')
p_df=spark.read.parquet('parquet_delay_basic')


In [48]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('temporary_parquet')


In [52]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.
sql="""
SELECT view,
ROUND(AVG(price), 2) AS avg_price
FROM Home_sales_df
GROUP BY view HAVING AVG(price) >=350000 view DESC.show
ORDER BY view;
start_time = time.time()

"""

print("--- %s seconds ---" % (time.time() - start_time))

--- 25.024728298187256 seconds ---


In [53]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("Home_sales_df")

In [54]:

# 15. Check if the home_sales_df is no longer cached
spark.catalog.isCached("Home_sales_df")





False